<h1> GARCH Bayesian Estimation Workflow

### GARCH(1,1) Model

The conditional variance in a GARCH(1,1) model is expressed as:

$
\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2
\$


### Constraints:
1. Stationarity:$\alpha + \beta < 1\$

2. Non-Negativity of Variance:$\omega > 0, \quad \alpha > 0, \quad \beta > 0\$


<h1> Load and Prepare Data

In [ ]:
data = pd.read_excel('Data.xlsx', sheet_name='GARCH')
Date = data.iloc[4:, 0].values  # Extract date column
stock_return = data.iloc[4:, 1].values  # Extract stock return column

mask = ~np.isnan(stock_return)
Y = stock_return[mask]
Date = Date[mask]

<h1> Initial Parameter set up

In [ ]:
 # Specifications
 # 
    Spec = {
        "MH": 1,
        "nu": 30,
        "theta_": np.array([0.01, 0.05, 0.90]),  # This vector specifies the prior mean for the GARCH parameters (𝜔,𝛼,𝛽)
        "thetav_": 0.0025 * np.ones(3), # This vector specifies the variance (or uncertainty) of the prior distributions for the GARCH parameters.
        "Y": Y,
    }

    theta_hat = np.array([0.01, 0.05, 0.90, np.log(0.01 / (1 - 0.05 - 0.90))])
    V = np.eye(len(theta_hat)) * 0.0025

<h1> Run MCMC

In [ ]:
n0, n1, freq = 2000, 20000, 500
    MHm, acceptance_rate = MCMC(lnpost, paramconst, n0, n1, theta_hat, V, freq, Spec)

    print(f"Acceptance Rate: {acceptance_rate:.2%}")

<h1> Simulate 

In [ ]:
# Simulate volatility and forecasts
    Volm = np.zeros((n1, len(Y)))
    Yfm = np.zeros(n1)

    for iter in range(n1):
        theta = MHm[iter, :]
        _, Vol = Kalman(theta, Y)
        Volm[iter, :] = Vol
        Yfm[iter] = Gen_Forecast(theta, Y, Vol)


<h1> VaR and Expected Shortfall

In [ ]:
    VaR = np.quantile(Yfm, 0.05)
    ES = np.mean(Yfm[Yfm < VaR])
    print(f"VaR: {VaR:.4f}, ES: {ES:.4f}")

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(np.abs(Y), label='Absolute Return')
    plt.plot(Volm.mean(axis=0), label='Volatility')
    plt.legend()
    plt.title("Volatility and Absolute Return")
    plt.show()

    return MHm, Volm, Yfm


<h1> Functions used above

<h1> Helper functions

In [ ]:
# minc: Computes the minimum value of an array.

def minc(arr):
    return np.min(arr)

# sumc: Computes the sum of an array.

def sumc(arr):
    return np.sum(arr)

# lnpdfn: Computes the log of the normal distribution's PDF.

def lnpdfn(x, mean, var):
    return -0.5 * np.log(2 * np.pi * var) - 0.5 * ((x - mean) ** 2) / var


<h1> Parameter constraints

In [ ]:
# Ensures parameters satisfy GARCH constraints
#
# 1. Parameters must be finite.
# 2. 𝜔,𝛼,𝛽>0
# 3. 𝛼+𝛽<0.99

def paramconst(theta, Spec):
    validm = np.ones(30)  # Assume all constraints are valid initially

    isfin = np.isfinite(theta)
    validm[0] = minc(isfin) > 0.5  # Check if all parameters are finite

    if validm[0] > 0:  # If parameters are finite
        validm[1] = minc(theta[:3]) > 0  # Check positivity of ω, α, β
        validm[2] = theta[1] + theta[2] < 0.99  # Stationarity condition

    valid = minc(validm)  # If any condition fails, valid = 0
    return valid, validm

<h1> Log-prior, Log-likelihood, Lost-posterior</h1>

In [ ]:
# lnprior: Encodes prior beliefs about the parameters:
# Normal priors for 𝜔,𝛼,𝛽
# A prior for the log of the long-run variance (eL2).

def lnprior(theta, Spec):
    theta_ = Spec["theta_"]  # Prior mean
    thetav_ = Spec["thetav_"]  # Prior variance

    # Prior for ω, α, β
    priorj = lnpdfn(theta[:3], theta_, thetav_).sum()

    # Prior for long-run variance
    eL2_mean = theta[0] / (1 - theta[1] - theta[2])  # Long-run variance
    priorj += lnpdfn(theta[3], np.log(eL2_mean), 0.0001)
    return priorj


# lnpost: Combines log-likelihood and log-prior to compute the log-posterior (lnpost(𝜃)=lnlik(𝜃)+lnprior(𝜃))

def lnpost(theta, Spec):
    lnlik0 = lnlik(theta, Spec)  # Log-likelihood
    lnprior0 = lnprior(theta, Spec)  # Log-prior
    return lnlik0 + lnprior0


# lnlik: Uses the Kalman filter to compute the likelihood of observing the data given the parameters.

def lnlik(theta, Spec):
    Y = Spec["Y"]  # Observed data
    lnL, _ = Kalman(theta, Y)  # Log-likelihood from Kalman filter
    return lnL

# Kalman Filter here simulates GARCH volatility and computes the log-likelihood for a time series.
# The Kalman filter is often used in state-space models, which represent the system in terms of latent states. 
# In this context, the GARCH model's variance can be treated as a latent state that evolves over time.
# The Kalman filter helps estimate the latent state, compute the log-likelihood. 


def Kalman(theta, ym):
    T = len(ym)
    a0, a1, gam1 = theta[:3]
    e_L2 = np.exp(theta[3])

    lnLm = np.zeros(T)  # Log-likelihood components
    Volm = np.zeros(T)  # Volatility components

    sig2_L = a0 / (1 - a1 - gam1)  # Long-run variance
    e_L = np.sqrt(e_L2)

    for t in range(T):
        yt = ym[t]
        sig2t = a0 + a1 * e_L**2 + gam1 * sig2_L  # Conditional variance
        lnLm[t] = lnpdfn(yt, 0, sig2t)  # Log-likelihood contribution
        Volm[t] = np.sqrt(sig2t)
        e_L = yt  # Update residual
        sig2_L = sig2t  # Update variance

    lnL = sumc(lnLm)
    return lnL, Volm

